In [57]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [58]:
import pandas as pd

train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"

df_train = pd.read_csv(train_path)

df_train.drop(columns=['user_id', 'book_id', 'date_added', 'date_updated',
                            'read_at', 'started_at', 'n_votes', 'n_comments'], inplace=True)

In [59]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stopwords_english = stopwords.words('english')

nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def preprocess_review(text):
    text = text.lower()

    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'(view spoiler|hide spoiler)', '', text)

    tokens = text.split()

    clean_tokens = [WordNetLemmatizer().lemmatize(tok) for tok in tokens if
                     tok not in stopwords_english and len(tok) > 1]
    # # clean_tokens = [tok for tok in tokens if tok not in stopwords_english and len(tok) > 1]

    clean_text = ' '.join(clean_tokens)

    return clean_text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\enzol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [60]:
df_train["clean_text"] = df_train["review_text"].apply(preprocess_review)

In [61]:
from sklearn.model_selection import train_test_split

x_train = df_train["clean_text"].values
y_train = df_train["rating"].values

x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

print("Training data:", x_tr.shape, y_tr.shape)
print("Validation data:", x_va.shape, y_va.shape)

Training data: (720000,) (720000,)
Validation data: (180000,) (180000,)


In [62]:
caracteres = "abcdefghijklmnopqrstuvwxyz "

VOCAB_SIZE = 30
MAX_SEQ_LENGTH = 250
EMBEDDING_DIMS = 128

In [63]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=VOCAB_SIZE, char_level=True, filters=caracteres)
tokenizer.fit_on_texts(x_tr)

x_tr_seq = tokenizer.texts_to_sequences(x_tr)
x_tr_seq = pad_sequences(x_tr_seq, maxlen=MAX_SEQ_LENGTH)

x_va_seq = tokenizer.texts_to_sequences(x_va)
x_va_seq = pad_sequences(x_va_seq, maxlen=MAX_SEQ_LENGTH)

In [64]:
x_tr[0]

'e review'

In [65]:
x_tr.shape

(720000,)

In [66]:
x_tr_seq[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [67]:
x_tr_seq.shape

(720000, 250)

In [68]:
epochs = 100

model_nb = 2
hidden_layer = 2
learning_rate = 1e-3
batch_size = 1024
dropout_rate = 0.4

In [69]:
from keras.models import Model
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout, Input
from keras.optimizers import Adam

input_text = Input(shape=(MAX_SEQ_LENGTH,))

embedding_layer = Embedding(VOCAB_SIZE, output_dim=EMBEDDING_DIMS, input_length=MAX_SEQ_LENGTH)(input_text)

x = embedding_layer

for _ in range(hidden_layer):
    x = Bidirectional(LSTM(units=int(EMBEDDING_DIMS//2), dropout=dropout_rate, return_sequences=True))(x)

last = Bidirectional(LSTM(units=EMBEDDING_DIMS, dropout=dropout_rate))(x)
# model.add(BatchNormalization())
# model.add(Dropout(dropout_rate))

dense = Dense(units=32, activation='relu')(last)
# model.add(BatchNormalization())
# model.add(Dropout(dropout_rate))

output = Dense(units=6, activation='softmax')(dense)

rnn_model = Model(input_text, output)

rnn_model.summary()

rnn_model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 250)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 250, 128)          3840      
                                                                 
 bidirectional_12 (Bidirecti  (None, 250, 128)         98816     
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 250, 128)         98816     
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 256)              263168    
 onal)                                                           
                                                           

In [70]:
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

earlystopping_cb = EarlyStopping(patience=6, restore_best_weights=True)
tensorboard = TensorBoard(f"logs/char/rnn_model_{model_nb}_hidden_layers_{hidden_layer}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}")

In [ ]:
rnn_model.fit(x_tr_seq,
                    y_tr,
                    validation_data=(x_va_seq, y_va),
                    callbacks=[tensorboard],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    )

Epoch 1/100
704/704 [==============================] - 165s 227ms/step - loss: 1.4554 - accuracy: 0.3645 - val_loss: 1.4415 - val_accuracy: 0.3724
Epoch 2/100
704/704 [==============================] - 158s 225ms/step - loss: 1.4169 - accuracy: 0.3878 - val_loss: 1.3967 - val_accuracy: 0.3961
Epoch 3/100
704/704 [==============================] - 155s 220ms/step - loss: 1.3926 - accuracy: 0.4013 - val_loss: 1.3677 - val_accuracy: 0.4140
Epoch 4/100
704/704 [==============================] - 155s 220ms/step - loss: 1.3546 - accuracy: 0.4175 - val_loss: 1.3304 - val_accuracy: 0.4253
Epoch 5/100
704/704 [==============================] - 155s 220ms/step - loss: 1.3163 - accuracy: 0.4331 - val_loss: 1.2873 - val_accuracy: 0.4432
Epoch 6/100
704/704 [==============================] - 158s 224ms/step - loss: 1.2818 - accuracy: 0.4477 - val_loss: 1.2527 - val_accuracy: 0.4582
Epoch 7/100
704/704 [==============================] - 156s 222ms/step - loss: 1.2545 - accuracy: 0.4589 - val_loss: 1

In [ ]:
df_test = pd.read_csv(result_path, sep=",")

df_test.sample()

In [ ]:
df_test = df_test.drop(columns=['user_id', 'book_id', 'date_added', 'date_updated',
                                'read_at', 'started_at', 'n_votes', 'n_comments'])

df_test["clean_text"] = df_test["review_text"].apply(preprocess_review)

df_test = df_test.drop(columns=['review_text'])

df_test.head()

In [ ]:
x_te = df_test["clean_text"].values

x_te_seq = tokenizer.texts_to_sequences(x_te)
x_te_seq = pad_sequences(x_te_seq, maxlen=MAX_SEQ_LENGTH)

In [ ]:
import numpy as np

predictions = [np.argmax(i) for i in rnn_model.predict(x_te_seq)]

# Create a new DataFrame to merge review ids and the model predictions
submission = pd.DataFrame({'review_id': df_test.review_id, 'rating': predictions})

# Check few random entries
submission.sample(10)

In [ ]:
submission.to_csv("res_files/submission.csv", index=None)

In [ ]:
test_loss, test_acc = rnn_model.evaluate(x_va_seq,y_va)

print('Validation loss:', test_loss)
print('Validation accuracy:', test_acc)